# Getting Started with the Arize Platform - Comparing Business Impact

**In this walkthrough, we are going to look at how to use  Arize to measure business impact in production!**

You manage the default/fraud detection model for the widely used [Lending Club](https://www.lendingclub.com/). One of your model is already in production and serving customers, but now you have a newly fitted model that you would like to compare with the production model.

Specifically, **you want to see which model has better impact on your business objectives in production**, so you turn to Arize for investigation. We will explain what "Business Impact" means when we get to Step 2: Understanding Business Impact



## Our steps to resolving this issue will be :

1. Get our model onto the Arize platform to investigate
2. Enter formula for evaluating business impact
3. Compare the model in production and the new model we want to validate

# Step 1: Setup and Getting the Data
### Step 1.1: Loading data from Arize client
We will load in some pre-existing data for the Lending Club model - training data, features, predictions, and class probabilities.

In [ ]:
import pandas as pd
import numpy as np


# 1 Load data from our model already in production
production = pd.read_csv(
    "https://storage.googleapis.com/arize-assets/fixtures/impact-production.csv"
).iloc[:, 1:]

# 2 Load validation data for our new model
validation = pd.read_csv(
    "https://storage.googleapis.com/arize-assets/fixtures/impact-validation.csv"
).iloc[:, 1:]

# 3 Unpack data so we can log them to Arize later!
def unpack_data(data):
    X, y = data.drop(columns=["label", "prediction", "score"]), data["label"]
    pred, score = data["prediction"], data["score"]
    return X, y, pred, score


X_val, y_val, y_val_pred, y_val_score = unpack_data(validation)
X_prod, y_prod, y_prod_pred, y_prod_score = unpack_data(production)

### Step 1.2: Setting up Arize Client:
First, copy the Arize `API_KEY` and `SPACE_KEY` from your admin page shown below!

<img src="https://storage.googleapis.com/arize-assets/fixtures/copy-keys.png" width="700">

In [ ]:
!pip install -q arize
from arize.pandas.logger import Client, Schema
from arize.utils.types import ModelTypes, Environments

# Step 1: Set-up Arize Client and model meta data
SPACE_KEY = "SPACE_KEY"
API_KEY = "API_KEY"
arize_client = Client(space_key=SPACE_KEY, api_key=API_KEY)

model_id = "business-impact-comparison"
model_type = ModelTypes.SCORE_CATEGORICAL

if SPACE_KEY == "SPACE_KEY" or API_KEY == "API_KEY":
    raise ValueError("❌ NEED TO CHANGE SPACE AND/OR API_KEY")
else:
    print("✅ Arize setup complete!")

We'll use the following helper functions to generate prediction IDs and timestamps to simulate a production environment.

In [ ]:
import uuid
from datetime import datetime, timedelta

# Prediction ID is required for logging any dataset
def generate_prediction_ids(X):
    return pd.Series((str(uuid.uuid4()) for _ in range(len(X))), index=X.index)


# OPTIONAL: We can directly specify when inferences were made
def simulate_production_timestamps(X, days=30):
    t = datetime.now()
    current_t, earlier_t = t.timestamp(), (t - timedelta(days=days)).timestamp()
    return pd.Series(np.linspace(earlier_t, current_t, num=len(X)), index=X.index)

### Step 1.3: Logging Model Version 1 (Production) to Arize
First, we take our existing model that's serving in production (Model version 1.0) and use the Arize SDK to log our predictions, actuals, and features. For more details on how **`arize.pandas.log`** works, visit out documentations page below.

[![Buttons_OpenOrange.png](https://storage.googleapis.com/arize-assets/fixtures/Buttons_OpenOrange.png)](https://docs.arize.com/arize/sdks-and-integrations/python-sdk/arize.pandas)

In [ ]:
# In the example here we need to first assemble our data into a pandas DataFrame
production_dataset = X_prod.join(
    pd.DataFrame(
        {
            "prediction_id": generate_prediction_ids(X_prod),
            "prediction_ts": simulate_production_timestamps(X_prod),
            "prediction_labels": y_prod_pred,
            "prediction_scores": y_prod_score,
            "actual_labels": y_prod,
        }
    )
)

Three easy steps to log a `pandas.DataFrame`. See [docs](https://docs.arize.com/arize/api-reference/python-sdk/arize.pandas) for more details.

1.   Define `Schema` to designate column names
2.   Call `arize.pandas.log()`
3.   Check `response.status_code`

In [ ]:
# Define a Schema() object for Arize to pick up data from the correct columns for logging
production_schema = Schema(
    prediction_id_column_name="prediction_id",  # REQUIRED
    timestamp_column_name="prediction_ts",
    prediction_label_column_name="prediction_labels",
    prediction_score_column_name="prediction_scores",
    actual_label_column_name="actual_labels",
    feature_column_names=X_prod.columns,
)

# arize_client.log returns a Response object from Python's requests module
response = arize_client.log(
    dataframe=production_dataset,
    schema=production_schema,
    model_id=model_id,
    model_version="1.0",
    model_type=model_type,
    environment=Environments.PRODUCTION,
)

if response.status_code != 200:
    print(
        f"❌ logging failed with response code {response.status_code}, {response.text}"
    )
else:
    print(
        f"✅ You have successfully logged {len(production_dataset)} data points to Arize"
    )

### Step 1.4: Logging Model Version 2 (Validation) to Arize
Next, we log our Model version 2.0 as validation to the Arize platform. With both models logged, we will be able to compare their performance using the business impact page!

In [ ]:
# In the example here we need to first assemble our data into a pandas DataFrame
validation_dataset = X_val.join(
    pd.DataFrame(
        {
            "prediction_id": generate_prediction_ids(X_val),
            "prediction_labels": y_val_pred,
            "prediction_scores": y_val_score,
            "actual_labels": y_val,
        }
    )
)

Three easy steps to log a `pandas.DataFrame`. See [docs](https://docs.arize.com/arize/api-reference/python-sdk/arize.pandas) for more details.

1.   Define `Schema` to designate column names
2.   Call `arize.pandas.log()`
3.   Check `response.status_code`

In [ ]:
# Define a Schema() object for Arize to pick up data from the correct columns for logging
validation_schema = Schema(
    prediction_id_column_name="prediction_id",  # REQUIRED
    prediction_label_column_name="prediction_labels",
    prediction_score_column_name="prediction_scores",
    actual_label_column_name="actual_labels",
    feature_column_names=X_val.columns,
)

# arize_client.log returns a Response object from Python's requests module
response = arize_client.log(
    dataframe=validation_dataset,
    schema=validation_schema,
    model_id=model_id,
    model_version="2.0",
    model_type=model_type,
    environment=Environments.VALIDATION,
    batch_id="validation",  # distinct batch_id is REQUIRED for logging validation datasets
)

if response.status_code != 200:
    print(
        f"❌ logging failed with response code {response.status_code}, {response.text}"
    )
else:
    print(
        f"✅ You have successfully logged {len(validation_dataset)} data points to Arize"
    )

## Check Data Ingestion Information

Data will be available in the UI in about 10 minutes after it was received. If data from a new model is sent, the model will be reflected almost immediately in the Arize platform. However, you will not see data yet. To verify data has been sent correctly and is being processed, we recommend that you check our Data Ingestion tab. 

You will be able to see the predictions, actuals, and feature importances that have been sent in the last week, last day or last 30 minutes. 

An example view of the Data Ingestion tab from a model, when data is sent continuously over 30 minutes, is shown in the image below. 

<img src="https://storage.googleapis.com/arize-assets/fixtures/data-ingestion-tab.png" width="700">

## Coffee Time ☕️
Note that the Arize performs takes about 10 minutes to index the data. While the model should appear immediately, the data will not show up till the indexing is done. Feel free to go grab a cup of coffee as Arize works its magic! 🔮

Your Prediction Volume may look slightly different!

![image.png](https://storage.googleapis.com/arize-assets/fixtures/waiting-on-data.png)

Actual data will show up under **Model Health**. Once the number changes from **0 Actuals** to **Actuals** (with summary statistics listed in the drop-down), your production actuals will have been fully recorded on Arize!

![image.png](https://storage.googleapis.com/arize-assets/fixtures/waiting-on-actual-data.png)

# Step 2: Understanding Business Impact
Sometimes, metrics such as accuracy and recall can't perfectly describe the business objective the model is trying to serve.
### **The Fraud Detection Example**

In our toy example, we run a default/fraud detection model for the lending club. In this business model, there are financial outcomes that change depending on whether our our model predicted `true` or `false` on a model, in addition to whether those predictions were accurate. **See Business Impact Figure for an example of how this works!**

1. When we correct predict default (TP), we save **$10 profit** from admin fees.

2. When we incorrectly predict default (FP), we incure a **$500 cost** as a need to re-acquire a new customer

3. When we correctly predict no-default (TN), we make **$1000 profit** from customer life-time value.

4. When we incorrectly predict no-default (FN), we incure a **$300 cost** from the contract.

Thus, since our model's goal is to optimize for a business outcome, evaluating model performance with metrics such as accuracy or recall alone might not be enough! So, we turn to Arize for help!

###**Business Impact Figure**

![image.png](https://storage.googleapis.com/arize-assets/fixtures/business-impact-final.png)

###**For more explaination of how the payoff curve and business impact works, visit our Documentations Page.**

[![Buttons_OpenOrange.png](https://storage.googleapis.com/arize-assets/fixtures/Buttons_OpenOrange.png)](https://arize.gitbook.io/arize/platform-features/business-impact-draft)


# Step 3: Analyze Payoff Curve on Arize
## **The Business Impact Tab**
1. Once production data show up, click on **Business Impact** tab. Select **`fraud`** under **`Select Prediction Value`** dropdown.

2. Copy the following formula we determined for estimating business impact.

```10 * TP_COUNT + 1000 * TN_COUNT - 500 * FP_COUNT -  300 * FN_COUNT```

## **Video Demo 1: Customer Business Impact Formula**
After entering the formula, you should see a shift in the business impact curve like below.

![Button_Open.png](https://storage.googleapis.com/arize-assets/fixtures/business-impact-rec-1-v2.gif)

Based on this curve, we could see that the best decision boundary for our business outcome should be between 0.4 - 0.65 $Pr(X = \texttt{fraud})$ for our score categorical model!

## **Video Demo 2: Comparing Model Impact**
Now, we want to see which one of our model performs better in accordance to our business impact formula. Following the same steps:

1. Select **`fraud`** under **`Select Prediction Value`** dropdown.

2. Select **`2.0 validation `** under **`Compare against`** dropdown.

3. Copy the same formula we determined for estimating business impact.

![Button_Open.png](https://storage.googleapis.com/arize-assets/fixtures/business-impact-rec-2-v2.gif)

### Overview
Arize is an end-to-end ML observability and model monitoring platform. The platform is designed to help ML engineers and data science practitioners surface and fix issues with ML models in production faster with:
- Automated ML monitoring and model monitoring
- Workflows to troubleshoot model performance
- Real-time visualizations for model performance monitoring, data quality monitoring, and drift monitoring
- Model prediction cohort analysis
- Pre-deployment model validation
- Integrated model explainability

### Website
Visit Us At: https://arize.com/model-monitoring/

### Additional Resources
- [What is ML observability?](https://arize.com/what-is-ml-observability/)
- [Playbook to model monitoring in production](https://arize.com/the-playbook-to-monitor-your-models-performance-in-production/)
- [Using statistical distance metrics for ML monitoring and observability](https://arize.com/using-statistical-distance-metrics-for-machine-learning-observability/)
- [ML infrastructure tools for data preparation](https://arize.com/ml-infrastructure-tools-for-data-preparation/)
- [ML infrastructure tools for model building](https://arize.com/ml-infrastructure-tools-for-model-building/)
- [ML infrastructure tools for production](https://arize.com/ml-infrastructure-tools-for-production-part-1/)
- [ML infrastructure tools for model deployment and model serving](https://arize.com/ml-infrastructure-tools-for-production-part-2-model-deployment-and-serving/)
- [ML infrastructure tools for ML monitoring and observability](https://arize.com/ml-infrastructure-tools-ml-observability/)

Visit the [Arize Blog](https://arize.com/blog) and [Resource Center](https://arize.com/resource-hub/) for more resources on ML observability and model monitoring.
